In [55]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2, warn.conflicts = FALSE)
library(tsibble, warn.conflicts = FALSE)
library(zeallot, warn.conflicts = FALSE)
library(sf, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(factoextra, warn.conflicts = FALSE)
library(assertr, warn.conflicts = FALSE)

source("src/paths/paths.R")


In [3]:
path.base <- file.path(path.ds, "ARPA", "FVG")
path.stats <- file.path(path.base, "station_info.csv")
path.data <- file.path(path.base, "dataset")


In [56]:
load_metadata <- function() {
    vroom::vroom(path.stats,
        col_types = vroom::cols(
            station_code = "c",
            station_name = "c",
            station_kind = "c",
            lat = "d",
            lon = "d",
            elevation = "d"
        )
    ) |>
        rename(original_id = station_code) |>
        mutate(dataset_id = "ARPAFVG", state = "Friuli-Venezia Giulia") |>
        mutate(station_id = as.character(row_number())) |>
        collect() |> #|> name_stations() |> collect()
        assert(is_uniq, c(station_id, original_id))
}


In [48]:
load_data <- function() {
    # Ho valutato se pescare anche la T media per fare delle correzioni, ma meglio evitare mischiotti
    open_dataset(file.path(path.base, "dataset")) |>
        rename(original_id = stazione, T_MIN = `Temp. min gradi C`, T_MAX = `Temp. max gradi C`) |>
        select(T_MIN, T_MAX, original_id, date) |>
        mutate(original_id = cast(original_id, utf8())) |>
        filter(!(is.na(T_MIN) & is.na(T_MAX))) |>
        collect() |>
        pivot_longer(cols = c(T_MIN, T_MAX), names_to = "variable", values_to = "value") |>
        mutate(merged = FALSE)
}


In [57]:
load_daily_data.arpafvg <- function() {
    meta <- load_metadata()
    data <- load_data() |>
        left_join(meta |> select(original_id, station_id), by = "original_id") |>
        select(-original_id) |>
        as_arrow_table()
        # as_arrow_table2(data_schema)

    list("meta" = meta, "data" = data)
}


In [58]:
a <- load_daily_data.arpafvg()


In [59]:
a$meta

original_id,station_name,station_kind,lat,lon,elevation,dataset_id,state,station_id
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
103400,Barcis,syn,46.19320,12.57103,468,ARPAFVG,Friuli-Venezia Giulia,1
178200,Grado Meteo,syn,45.68035,13.37329,0,ARPAFVG,Friuli-Venezia Giulia,2
188100,Grado Marea,syn,45.68316,13.38343,0,ARPAFVG,Friuli-Venezia Giulia,3
231600,Musi 2,syn,46.30727,13.27258,600,ARPAFVG,Friuli-Venezia Giulia,4
ARI,Ariis,syn,45.87830,13.09000,13,ARPAFVG,Friuli-Venezia Giulia,5
BAR,Barcis (fino al 18/02/2021),syn,46.19320,12.57103,468,ARPAFVG,Friuli-Venezia Giulia,6
BGG,Borgo Grotta Gigante,syn,45.70938,13.76468,275,ARPAFVG,Friuli-Venezia Giulia,7
BIC,Bicinicco,syn,45.92881,13.24014,33,ARPAFVG,Friuli-Venezia Giulia,8
BOA,Boa Paloma (fino al 06/02/2018),syn,45.61829,13.56502,5,ARPAFVG,Friuli-Venezia Giulia,9
